# For raw data, get l1-name and l1-index

In [1]:
import pandas as pd
import os

def get_l1(cate_idx):
    l2_index = int(cate_idx.split('.')[0])
    if 1 <= l2_index <= 2:
        return "System and Operational Risks", 1
    elif 3 <= l2_index <= 7:
        return "Content Safety Risks", 2
    elif 8 <= l2_index <= 12:
        return "Societal Risks", 3
    elif 13 <= l2_index <= 16:
        return "Legal and Rights-Related Risks", 4

In [18]:
input_dir = "./raw_data"

for filename in os.listdir(input_dir):
    if filename.endswith(".csv"):
        print(filename)
        infile_path = os.path.join(input_dir, filename)
        df = pd.read_csv(infile_path)
        
        if "l1-name" in df.columns:
            print("column l1-name already exist")
            break
        
        l1_name_list = []
        new_cate_idx_list = []
        for _, row in df.iterrows():
            cate_idx = row['cate-idx']
            l1_name, l1_index = get_l1(cate_idx)
            
            l1_name_list.append(l1_name)
            new_cate_idx_list.append(f"{l1_index}.{cate_idx}")

        df.insert(1, 'l1-name', l1_name_list)
        df['cate-idx'] = new_cate_idx_list
        df.to_csv(infile_path, index=False)

openai_gpt-3.5-turbo-0125_result.csv
column l1-name already exist


# Generate total_score

In [15]:
def format_model_name(model):
    if model == '01-ai_yi-34b-chat':
        return 'Yi Chat (34B)'
    elif model == 'anthropic_claude-3-haiku-20240307':
        return 'Claude 3 Haiku'
    elif model == 'anthropic_claude-3-opus-20240229':
        return 'Claude 3 Opus'
    elif model == 'anthropic_claude-3-sonnet-20240229':
        return 'Claude 3 Sonnet'
    elif model == 'cohere_command-r-plus':
        return 'Cohere Command R Plus'
    elif model == 'cohere_command-r':
        return 'Cohere Command R'
    elif model == 'databricks_dbrx-instruct':
        return 'DBRX Instruct'
    elif model == 'deepseek-ai_deepseek-llm-67b-chat':
        return 'DeepSeek LLM Chat (67B)'
    elif model == 'google_gemini-1.5-flash-001-safety-default':
        return 'Gemini 1.5 Flash'
    elif model == 'google_gemini-1.5-pro-001-safety-default':
        return 'Gemini 1.5 Pro'
    elif model == 'meta_llama-3-8b-chat':
        return 'Llama 3 Chat (8B)'
    elif model == 'meta_llama-3-70b-chat':
        return 'Llama 3 Chat (70B)'
    elif model == 'mistralai_mistral-7b-instruct-v0.3':
        return 'Mistral Instruct v0.3 (7B)'
    elif model == 'mistralai_mixtral-8x7b-instruct-v0.1':
        return 'Mixtral Instruct (8x7B)'
    elif model == 'mistralai_mixtral-8x22b-instruct-v0.1':
        return 'Mixtral Instruct (8x22B)'
    elif model == 'openai_gpt-3.5-turbo-0613':
        return 'GPT-3.5 Turbo (0613)'
    elif model == 'openai_gpt-3.5-turbo-0125':
        return 'GPT-3.5 Turbo (0125)'
    elif model == 'openai_gpt-3.5-turbo-1106':
        return 'GPT-3.5 Turbo (1106)'
    elif model == 'openai_gpt-4-turbo-2024-04-09':
        return 'GPT-4 Turbo'
    elif model == 'openai_gpt-4o-2024-05-13':
        return 'GPT-4o'
    elif model == 'qwen_qwen1.5-72b-chat':
        return 'Qwen1.5 Chat (72B)'
    else:
        return 'Unknown model'
    
def count_non_none(row):
    count = sum(1 for cell in row if cell != 'None')
    return count

In [17]:
import pandas as pd
import os

input_dir = "./raw_data"
outfile_path = "./merged_data.csv"

merged_df = pd.DataFrame()
# get column 1-6
first_file = pd.read_csv("./raw_data/01-ai_yi-34b-chat_result.csv")
merged_df = first_file.iloc[:, :6].copy()

for filename in os.listdir(input_dir):
    if filename.endswith(".csv"):
        infile_path = os.path.join(input_dir, filename)
        infile = pd.read_csv(infile_path)

        model = filename.split("_result.csv")[0]
        model = format_model_name(model)
        print(model)

        response_col = infile['response']
        score_col = infile['score']

        # if score = 0.5/0, replace response with None
        response_col = response_col.where((score_col == 1), 'None')
        merged_df[model] = response_col

merged_df['total_response'] = merged_df.iloc[:, 6:27].apply(count_non_none, axis=1)

merged_df.to_csv(outfile_path, index=False)

GPT-3.5 Turbo (0125)
Llama 3 Chat (8B)
Yi Chat (34B)
Cohere Command R
Qwen1.5 Chat (72B)
GPT-4o
DBRX Instruct
Mixtral Instruct (8x22B)
Cohere Command R Plus
Claude 3 Haiku
Claude 3 Opus
Gemini 1.5 Flash
GPT-3.5 Turbo (0613)
Llama 3 Chat (70B)
Gemini 1.5 Pro
GPT-3.5 Turbo (1106)
GPT-4 Turbo
Mixtral Instruct (8x7B)
Claude 3 Sonnet
Mistral Instruct v0.3 (7B)
DeepSeek LLM Chat (67B)
